In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np
import os

# Путь для сохранения данных и модели
data_path = '/home/roman/ds-phase-2/11-nn-ext/Проект/book_data2.csv'
index_path = '/home/roman/ds-phase-2/11-nn-ext/Проект/faiss_index.index'
model_name = 'cointegrated/rubert-tiny2'
embedding_dim = 0  # Инициализация переменной для размера эмбеддинга

# Загрузка данных
books_df = pd.read_csv(data_path)

# Фильтруем слишком короткие аннотации для улучшения качества поиска
books_df = books_df[books_df['annotation'].apply(lambda x: len(str(x)) > 50)]

# Используем модель ruBERT для создания эмбеддингов
model = SentenceTransformer(model_name)

# Генерация эмбеддингов для аннотаций
annotations = books_df['annotation'].tolist()
annotation_embeddings = model.encode(annotations, convert_to_tensor=True).cpu().numpy()
embedding_dim = annotation_embeddings.shape[1]  # Устанавливаем размерность эмбеддинга

# Создание и сохранение FAISS индекса для быстрого поиска
index = faiss.IndexFlatIP(embedding_dim)
index.add(annotation_embeddings)
faiss.write_index(index, index_path)  # Сохраняем индекс

print(f"FAISS индекс сохранён по пути: {index_path}")

def load_index():
    """Загружает FAISS индекс из файла."""
    if os.path.exists(index_path):
        loaded_index = faiss.read_index(index_path)
        print("FAISS индекс загружен успешно.")
        return loaded_index
    else:
        print("Ошибка: индекс не найден.")
        return None

# Загружаем FAISS индекс для поиска
index = load_index()

def search_books(query, top_k=5, mode='symmetric'):
    """
    Поиск по книгам на основе пользовательского запроса.
    
    Параметры:
        query (str): Описание книги, введенное пользователем.
        top_k (int): Количество возвращаемых результатов.
        mode (str): 'symmetric' для симметричного поиска или 'asymmetric' для асимметричного.
        
    Возвращает:
        pd.DataFrame: Результаты поиска с автором, названием, аннотацией и мерой близости.
    """
    # Генерация эмбеддинга для запроса и приведение к двумерному виду
    query_embedding = model.encode(query, convert_to_tensor=True).cpu().numpy().reshape(1, -1)
    
    # Поиск ближайших соседей
    if mode == 'symmetric':
        distances, indices = index.search(query_embedding, top_k * 2)  # Ищем больше записей для фильтрации
        distances = distances[0]
        indices = indices[0]
    else:  # Ассиметричный поиск
        distances = util.pytorch_cos_sim(query_embedding, annotation_embeddings)
        distances = distances.cpu().numpy().flatten()
        indices = np.argsort(-distances)[:top_k * 2]
        distances = distances[indices]
    
    # Подготовка результатов, исключая записи с NaN в нужных полях
    results = []
    count = 0  # Счётчик подходящих результатов
    
    for idx, score in zip(indices, distances):
        author = books_df.iloc[idx]['author']
        title = books_df.iloc[idx]['title']
        annotation = books_df.iloc[idx]['annotation']
        
        # Проверка на наличие автора и названия
        if pd.notna(author) and pd.notna(title):
            results.append({
                'author': author,
                'title': title,
                'annotation': annotation,
                'similarity_score': score
            })
            count += 1
            if count >= top_k:  # Останавливаем сбор, когда достигнуто необходимое количество
                break
    
    # Преобразование результатов в DataFrame
    results_df = pd.DataFrame(results)
    
    # Проверка на пустоту DataFrame и вывод содержимого для отладки
    if results_df.empty:
        print("Внимание: Нет подходящих результатов для данного запроса.")
    else:
        print("Результаты поиска:")
    
    return results_df

# Пример использования
query = "Книга о развитии"
top_k = 3
mode = 'asymmetric'  # Или 'asymmetric' для асимметричного поиска
results_df = search_books(query, top_k, mode)

# Выводим результаты, если они есть
if not results_df.empty:
    print(results_df[['author', 'title', 'annotation', 'similarity_score']])


/home/roman/anaconda3/envs/text/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


FAISS индекс сохранён по пути: /home/roman/ds-phase-2/11-nn-ext/Проект/faiss_index.index
FAISS индекс загружен успешно.
Результаты поиска:
            author                                              title  \
0         Пруст М.                                    В сторону Свана   
1     Стейнбек Дж.                         О мышах и людях. Жемчужина   
2   Карамзин  Н.М.  Бедная Лиза. Подробный иллюстрированный коммен...   

                                          annotation  similarity_score  
0  «В сторону Свана» - это первая книга семитомно...          0.559260  
1  "О мышах и людях" — повесть, не выходящая из Т...          0.555117  
2  В издании представлена повесть Н.М. Карамзина ...          0.552815  
